In [3]:
import os
import requests
# 【硬性要求】加载环境变量 (自动读取 .env 中的 API Keys)
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool

# ==========================================
# 1. 构建真实的 FlightAware API 工具 
# ==========================================
@tool
def get_flight_status(flight_id: str) -> str:
    """
    获取指定航班的实时航班状态、计划起飞时间和到达时间。
    输入必须是航班号，例如 'EK524' 或 'UAL1'。
    """
    # 获取环境变量中的真实 API Key
    api_key = os.getenv("FLIGHTAWARE_API_KEY")
    if not api_key:
        return "Error: FLIGHTAWARE_API_KEY not found in environment variables."
        
    print(f"\n[后台物理调用] 📡 正在连接 FlightAware AeroAPI 查询航班: {flight_id}...")
    
    # FlightAware AeroAPI (v4) 的标准请求端点
    url = f"https://aeroapi.flightaware.com/aeroapi/flights/{flight_id}"
    headers = {
        "x-apikey": api_key,
        "Accept": "application/json"
    }
    
    try:
        # 发起真实的 HTTP GET 请求
        response = requests.get(url, headers=headers)
        response.raise_for_status() # 如果返回 401(未授权) 或 404(未找到)，会自动抛出异常
        
        # 将返回的 JSON 响应转化为 Python 字典
        data = response.json()
        
        # 提取最新一条航班记录的核心数据 (数据清洗)
        if "flights" in data and len(data["flights"]) > 0:
            flight = data["flights"][0]
            status = flight.get("status", "Unknown")
            # API 返回的时间通常是 ISO 8601 格式的 UTC 时间，例如 "2024-05-01T10:00:00Z"
            departure = flight.get("scheduled_out", "Unknown")
            arrival = flight.get("scheduled_in", "Unknown")
            
            # 将清洗后的干净字符串返回给大模型上下文
            result = f"Flight {flight_id} Status: {status}. Scheduled Departure: {departure}. Scheduled Arrival: {arrival}."
            print(f"[API 成功返回] 🟢 {result}")
            return result
        else:
            return f"No active live data found for flight {flight_id} at this moment."
            
    except requests.exceptions.RequestException as e:
        return f"API network error or invalid API Key: {str(e)}"

# ==========================================
# 2. 组装终极智能体 (Assemble the Ultimate Agent)
# ==========================================
# 工具箱：真实航班 API + Python 数学计算器
tools = [get_flight_status, PythonREPLTool()]

# 使用强大的 GPT-4o 作为中央处理器
llm = ChatOpenAI(model="gpt-4o", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an elite travel assistant. You can check REAL-TIME flight status and use Python to perform strictly accurate time calculations. Note that flight API times are usually in UTC ISO format. Always use Python for math."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 创建现代化的工具调用 Agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# ==========================================
# 3. 真实数据实战挑战 (Live Execution)
# ==========================================
if __name__ == "__main__":
    # 使用老师的最爱 EK524，或者你可以换成你最近正在关注的真实航班号（比如 UAL1 代表美联航 1 号航班）
    target_flight = "UAL1" 
    
    complex_query = f"Check the real-time arrival time for flight {target_flight}. If I need to wait at the airport for exactly 3.5 hours after its scheduled arrival, what exact time should I book my taxi? Please use Python to calculate the final time."
    
    print(f"=== 🙋 发起真实世界极限挑战: {complex_query} ===")
    print("🔥 见证奇迹的时刻：大模型将自行抓取真实网络数据，并自己写 Python 算时间！\n")
    
    try:
        response = agent_executor.invoke({"input": complex_query})
        print(f"\n=== 🤖 最终完美回答 ===")
        print(response['output'])
    except Exception as e:
        print(f"\n❌ 运行异常，请检查你的 API 密钥额度或网络配置: {e}")

=== 🙋 发起真实世界极限挑战: Check the real-time arrival time for flight UAL1. If I need to wait at the airport for exactly 3.5 hours after its scheduled arrival, what exact time should I book my taxi? Please use Python to calculate the final time. ===
🔥 见证奇迹的时刻：大模型将自行抓取真实网络数据，并自己写 Python 算时间！



> Entering new AgentExecutor chain...

Invoking: `get_flight_status` with `{'flight_id': 'UAL1'}`



[后台物理调用] 📡 正在连接 FlightAware AeroAPI 查询航班: UAL1...
[API 成功返回] 🟢 Flight UAL1 Status: Scheduled. Scheduled Departure: 2026-02-28T06:35:00Z. Scheduled Arrival: 2026-03-01T00:01:00Z.
Flight UAL1 Status: Scheduled. Scheduled Departure: 2026-02-28T06:35:00Z. Scheduled Arrival: 2026-03-01T00:01:00Z.
Invoking: `Python_REPL` with `{'query': "from datetime import datetime, timedelta\n\n# Scheduled arrival time in UTC\nscheduled_arrival_str = '2026-03-01T00:01:00Z'\n\n# Convert to datetime object\nscheduled_arrival = datetime.fromisoformat(scheduled_arrival_str.replace('Z', '+00:00'))\n\n# Add 3.5 hours to the schedu